In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = pd.read_csv('kc_house_data.csv')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed.

In [6]:
train_data, test_data = train_test_split(sales, train_size=0.8, random_state=0)

# Learning a multiple regression model

In [8]:
from sklearn.linear_model import LinearRegression

In [9]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
lr = LinearRegression()
example_model = lr.fit(train_data[example_features], train_data['price'])

In [11]:
print example_model.coef_, example_model.intercept_

[   313.17055038 -56754.66651422   6887.71910816] 67512.0151381


In [13]:
example_predictions = example_model.predict(train_data[example_features])
print example_predictions[0] # should be 271789.505878

395813.498803


# Compute RSS

In [14]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)
    # Then compute the residuals/errors
    residuals = outcome - predictions
    # Then square and add them up
    RSS = np.dot(residuals, residuals)
    return(RSS)    

In [17]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data[example_features], test_data['price'])
print rss_example_train # should be 2.7376153833e+14

2.59213572106e+14


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [18]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [19]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [20]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

train_data['bed_bath_rooms'] = train_data['bedrooms'] * train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms'] * test_data['bathrooms']

train_data['lat_plus_long'] = train_data['lat'] + train_data['long']
test_data['lat_plus_long'] = test_data['lat'] + test_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [21]:
print np.mean(test_data['bedrooms_squared'])
print np.mean(test_data['bed_bath_rooms'])
print np.mean(test_data['log_sqft_living'])
print np.mean(test_data['lat_plus_long'])

12.2105019662
7.44772148971
7.55023946779
-74.6542606986


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [22]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [24]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = LinearRegression()
model_2 = LinearRegression()
model_3 = LinearRegression()
model_1 = model_1.fit(train_data[model_1_features], train_data['price'])
model_2 = model_2.fit(train_data[model_2_features], train_data['price'])
model_3 = model_3.fit(train_data[model_3_features], train_data['price'])

In [25]:
# Examine/extract each model's coefficients:
coef1 = model_1.coef_
print 'coef1: '
print coef1
coef2 = model_2.coef_
print 'coef2: '
print coef2
coef3 = model_3.coef_
print 'coef3: '
print coef3

coef1: 
[  3.12942010e+02  -5.30962691e+04   1.47770428e+04   6.53983343e+05
  -3.25707336e+05]
coef2: 
[  3.06819573e+02  -1.04604718e+05  -7.01815289e+04   6.50590952e+05
  -3.09965751e+05   2.49441497e+04]
coef3: 
[  5.37808086e+02   2.78047910e+03   1.01363766e+05   5.30798406e+05
  -4.09655435e+05  -1.81822552e+04   7.24579939e+02  -5.71030023e+05
   1.21142971e+05]


# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

In [27]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
rss1 = get_residual_sum_of_squares(model_1, train_data[model_1_features], train_data['price'])
rss2 = get_residual_sum_of_squares(model_2, train_data[model_2_features], train_data['price'])
rss3 = get_residual_sum_of_squares(model_3, train_data[model_3_features], train_data['price'])
print "rss1: " + str(rss1)
print "rss2: " + str(rss2)
print "rss3: " + str(rss3)

rss1: 9.79843597588e+14
rss2: 9.7079919973e+14
rss3: 9.13653644975e+14


Now compute the RSS on on TEST data for each of the three models.

In [28]:
# Compute the RSS on TESTING data for each of the three models and record the values:
trss1 = get_residual_sum_of_squares(model_1, test_data[model_1_features], test_data['price'])
trss2 = get_residual_sum_of_squares(model_2, test_data[model_2_features], test_data['price'])
trss3 = get_residual_sum_of_squares(model_3, test_data[model_3_features], test_data['price'])
print "trss1: " + str(trss1)
print "trss2: " + str(trss2)
print "trss3: " + str(trss3)

trss1: 2.13487129319e+14
trss2: 2.10778544169e+14
trss3: 2.03972051918e+14
